In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os
import sys
os.chdir('..')
sys.path.append('src')
os.environ['KMP_DUPLICATE_LIB_OK']='True'

In [3]:
from pathlib import Path
from matplotlib import pyplot as plt
import cv2
import tqdm
from ipywidgets import widgets
from IPython.display import clear_output
import rasterio
import functools
import numpy as np
from PIL import Image

import utils
from sampler import BackgroundSampler, GdalSampler
from lightning import cut_off_light_from_img
from rle2tiff import save_mask
from tf_reader import TFReader
from skimage.transform import resize

import functools
import numpy as np

#### Configurations

In [4]:
path_to_img_folder = Path('F:/kaggle/newdata/tiff_light')
path_to_mask_folder = Path('F:/kaggle/newdata/tiff_light/masks')
path_to_anot_struct_folder = Path('F:/kaggle/newdata/tiff_light')
postfix_anot = '-anatomical-structure.json'
postfix_polyg = '.json'
out_path_to_light_crop = Path('F:/kaggle/newdata/data/crop_aa05346ff/')

num_samples = 30 # Total number of lightning templates (backs)
wh = (768, 768)
downscale_factor = 3
max_gray_val = 255

rand_shift = wh[0]/2 # only for GdalSampler
sampler_type = 'back'   # glom or back

#### Create background samples 

Choose only images with lightnings

In [5]:
path_to_imgs = sorted(list(path_to_img_folder.glob('*.tiff')))
print(path_to_imgs)
indices_img_with_light = [1]  # Chosen manually
assert len(indices_img_with_light) == 1

path_to_imgs = [path_to_imgs[ind] for ind in indices_img_with_light]
print(path_to_imgs[0])

[WindowsPath('F:/kaggle/newdata/tiff_light/4ef6695ce.tiff'), WindowsPath('F:/kaggle/newdata/tiff_light/aa05346ff.tiff'), WindowsPath('F:/kaggle/newdata/tiff_light/c68fe75ea.tiff'), WindowsPath('F:/kaggle/newdata/tiff_light/e79de561c.tiff')]
F:\kaggle\newdata\tiff_light\aa05346ff.tiff


In [6]:
threshold=205
kernel_size=2

out_path_to_light_crop.mkdir(exist_ok=True) 

for path_to_img in path_to_imgs:
    path_to_mask =path_to_mask_folder / path_to_img.name
    path_to_anot_struct = path_to_anot_struct_folder / (path_to_img.stem + postfix_anot)
    path_to_polyg = path_to_anot_struct_folder / (path_to_img.stem + postfix_polyg)

    num_samples_per_img = num_samples//len(path_to_imgs)
    
    # List of polygons containing anotomical structure
    polygons = utils.get_cortex_polygons(utils.jread(path_to_anot_struct))
    if sampler_type == 'back':
        samples = BackgroundSampler(path_to_img, 
                                         path_to_mask, 
                                         polygons, 
                                         wh, 
                                         num_samples_per_img)
    elif sampler_type == 'glom':
        samples = GdalSampler(path_to_img, 
                              path_to_mask, 
                              path_to_polyg, 
                              wh, 
                              rand_shift_range = (rand_shift, rand_shift))
        
    
    for ind, back_sample in tqdm.tqdm(enumerate(samples)):
        crop, _ = back_sample
        num_bands = crop.shape[0]
        
        mask_with_light = cut_off_light_from_img(crop,
                                                 threshold=threshold,
                                                 kernel_size=kernel_size, 
                                                 min_light_ratio = 0.1, 
                                                 max_light_ratio = 0.35,
                                                 is_morph_before_thresh = False, 
                                                 is_morph_after_thresh = False)
        
        if mask_with_light is not None:
            # Reshape 
            h, w = mask_with_light.shape
            mask_with_light = mask_with_light.reshape(1, h, w)        # (1, h, w)
            
            # Downscale
            h_down, w_down = h//downscale_factor, w//downscale_factor
            mask_with_light = (resize(mask_with_light, (1, h_down, w_down)) * max_gray_val).astype(np.uint8)
            mask_with_light = np.where(mask_with_light > 127, max_gray_val, 0).astype(np.uint8)
            crop = (resize(crop, (num_bands, h_down, w_down)) * max_gray_val).astype(np.uint8)
            
            # Stack
            crop_with_mask = np.concatenate((crop, mask_with_light))  # (4, h, w)
            
            # Save
            path_to_save = out_path_to_light_crop / f'{path_to_img.stem}_{ind}{path_to_img.suffix}'
            utils.save_arr_as_tiff(crop_with_mask, path_to_save)
            
    del samples, crop

D:\Users\Mikhail\anaconda3\envs\Soft\lib\site-packages\rasterio\__init__.py:219: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  s = DatasetReader(path, driver=driver, sharing=sharing, **kwargs)
0it [00:00, ?it/s]D:\Users\Mikhail\anaconda3\envs\Soft\lib\site-packages\rasterio\__init__.py:229: NotGeoreferencedWarning: Dataset has no geotransform set. The identity matrix may be returned.
  **kwargs)
30it [00:10,  2.86it/s]


#### Clean

In [7]:
path_to_crops = out_path_to_light_crop.glob(f'*{path_to_imgs[0].stem}*{path_to_imgs[0].suffix}')
plt.rcParams["figure.figsize"] = (12,6)

del_mem_paths = []

def delete(light_crop, nothing):
    if light_crop not in del_mem_paths:
        del_mem_paths.append(light_crop)
        print("Added")
    
def f(nothing):
    clear_output(wait=True)
    display(button)
    
    try:
        
        path_to_light_crop = next(path_to_crops)
        print(path_to_light_crop)
        
        display(button_del)
        button_del._click_handlers.callbacks = []
        button_del.on_click(functools.partial(delete, str(path_to_light_crop)))
        
        dt_crop_with_light = TFReader(path_to_light_crop)
        crop_with_light = dt_crop_with_light.read()
        
        mask = crop_with_light[3]
        
        total_num_cells = 256*256
        num_white_cells = mask.sum() / max_gray_val
        ratio = num_white_cells / total_num_cells
        print(ratio)
        
        fig, (ax1, ax2) = plt.subplots(1, 2)
        ax1.imshow(crop_with_light[:3].transpose([1, 2, 0]), cmap='gray')
        ax2.imshow(mask, cmap='gray')
        plt.show()

        del dt_crop_with_light

    except StopIteration:
        print("ATTENTION: DELETE IMAGES")

    
button = widgets.Button(description="Next")
button_del = widgets.Button(description="Delete")

display(button)
button.on_click(f)

Button(description='Next', style=ButtonStyle())

ATTENTION: DELETE IMAGES


#### Delete

In [17]:
print(f'{len(np.unique(del_mem_paths))} crops will be removed')

0 crops will be removed


In [18]:
print(f'{len(np.unique(del_mem_paths))} crops has been removed')
for path in del_mem_paths:
    if os.path.isfile(Path(path)):
        Path(path).unlink()

0 crops has been removed


In [96]:
tiff_files = list((Path(out_path_to_light_crop)).glob('*.tiff'))

In [97]:
for tiff_file in tiff_files:
    img = rasterio.open(f"{tiff_file}", 'r')
    img_array = img.read()        
    h = img_array.shape[1]
    w = img_array.shape[2]
    img_array[3] = np.where(img_array[3] > 0, 255, 0).astype(np.uint8)
    name = tiff_file.with_suffix('').name
    img_tiff = rasterio.open(f"{out_path_to_light_crop}/png/{name}.png", 'w', driver='png', height=h, width=w, count=4, nbits=48, dtype=np.uint8)
    img_tiff.write(img_array)
    img_tiff.close()

In [58]:
#path = "F:/kaggle/newdata/data/light/e79de561c/"

In [59]:
#png_files = list((Path(path)).glob('*.png'))

In [60]:
#for png_file in png_files:
#    img = rasterio.open(f"{png_file}", 'r')
#    img_array = img.read()        
#    img_array = img_array[:, :256, :256]
#    h = img_array.shape[1]
#    w = img_array.shape[2]
#    name = png_file.with_suffix('').name
#    img_png = rasterio.open(f"{path}/256/{name}.png", 'w', driver='png', height=h, width=w, count=4, nbits=48, dtype=np.uint8)
#    img_png.write(img_array)
#    img_png.close()    